## Imports

In [1]:
## Read the config
# yml_path = '_quickstart/config_3d.yml'
from configs.get_config import get_config_from_yml

config = get_config_from_yml("/scratch/rhm4nj/cral/ginn/_quickstart/config_3d.yml")
config = get_config_from_yml("/scratch/rhm4nj/cral/ginn/configs/config_3d.yml")

In [2]:
!ml cuda/11.8.0

import os
## For setup
import torch
from configs.get_config import get_config_from_yml
from GINN.shape_boundary_helper import ShapeBoundaryHelper
from GINN.helpers.mp_manager import MPManager
from GINN.helpers.timer_helper import TimerHelper
# from GINN.morse.scc_surfacenet_manager import SCCSurfaceNetManager
from GINN.problem_sampler import ProblemSampler
from GINN.visualize.plotter_3d import Plotter3d
from train.train_utils.autoclip import AutoClip
from utils import get_model, get_stateless_net_with_partials

## For extracting and plotting a mesh
import k3d
from notebooks.notebook_utils import get_mesh_for_latent

## For running a training loop
import einops
from tqdm import trange
from models.model_utils import tensor_product_xz
from train.losses import closest_shape_diversity_loss, eikonal_loss, envelope_loss, interface_loss, normal_loss_euclidean, obstacle_interior_loss, strain_curvature_loss
from train.train_utils.latent_sampler import sample_new_z
from utils import set_all_seeds

In [3]:
from collections import Counter
FAILS = Counter()      

cuda_device = 0
yml_path = 'configs/config_3d.yml'
runs_dir = "all_runs"
save_interval = 10
num_epochs = 100
args = [yml_path, num_epochs]

# args
def get_file_name(args): 
    return '_'.join([str(i) for i in args]).replace('/', '_').replace('.', '_')

filename = get_file_name(args)
print(filename)

configs_config_3d_yml_100


In [4]:
from contextlib import nullcontext
import logging
import math

import torch
from tqdm import trange
from sklearn.cluster import DBSCAN
from GINN.helpers.mp_manager import MPManager

from models.point_wrapper import PointWrapper
from GINN.helpers.timer_helper import TimerHelper
from utils import do_plot, get_is_out_mask, precompute_sample_grid

logging.basicConfig(level=logging.INFO)

class CriticalPointsHelper():

    def __init__(self, config, netp, mp_manager: MPManager, plot_helper, timer_helper: TimerHelper, device):
        self.config = config
        ## Make the parameters not request gradient computation by creating a detached copy.
        ## The copy is shallow, meaning it shares memory, and is tied to parameter updates in the outer loop.
        self.netp = netp.detach()
        self.mpm = mp_manager
        
        self.logger = logging.getLogger('cp_helper')
        
        self.bounds = self.config['bounds']
        self.x_grid, self.x_grid_dist = precompute_sample_grid(self.config['n_points_find_cps'], self.bounds.to(device))
        self.x_grid_dist_squared_norm = self.x_grid_dist.square().sum()
        self.pc = None
        self.edges_z_list = None
        self.indexes = None
        self.p_surface = None
        
        self.plotter = plot_helper
        self.record_time = timer_helper.record if timer_helper is not None else nullcontext

    def get_critical_points(self, z, epoch, **kwargs):
        
        ## (-1) Reuse previous graph
        # reuse_output = self.reuse_graph()
        # if reuse_output is not None:
        #     return
        if epoch%self.config['recalc_cps_every_n_epochs']==0:
            ## Recompute the graph on initial epoch and every n epochs
            pass
        else:
            ## Reuse the graph
            self.logger.info(f'=== Reusing the graph ===')
            if self.pc is None or len(self.pc)==0 or self.edges_z_list is None or len(self.edges_z_list)==0:
                self.logger.info(f"(-1) FAIL: no valid pc or edges")
                ## If there are no previously known critical points, we cannot update them.
                ## We have two choices:
                ## (A) skip computing the graph and the scc_loss for the next (at most) recalc_cps_every_n_epochs
                FAILS[f"(-1) FAIL: no valid pc or edges"] += 1
                return False, None
                ## (B) try to compute the graph at every stage.
                ## This is cleaner, but more compute intense.
            
            with self.record_time('cp_helper: adjusting CPs'):
                pc_, _, x_path_over_iters = self._gd_to_min_grad_norm(z, self.pc, epoch, self.config['lr_adjust_cps'], early_stop=True, plot_descent=self.plotter.do_plot('plot_cp_descent_refinement'))
                print("post cp_helper", len(pc_))
            self.mpm.plot(self.plotter.plot_descent_trajectories, 'plot_cp_descent_refinement',
                                          [pc_.cpu().numpy(), x_path_over_iters.cpu().numpy()], 
                                          {'fig_label':'Trajectories of refining CPs', 'plot_anyway':True})

            is_update_valid = len(self.pc) == len(pc_)
            ## TODO: another condition is point consistency:
            ## 1) distance in geometric space should not exceed some maximum
            ## 2) same index
            self.pc = pc_
            if is_update_valid: ## check if the update is valid
                with torch.no_grad():
                    yc_u = self.netp.f(self.pc.data, self.pc.z_in(z)).squeeze(1)
                self.logger.info(f"(-1) SUCCESS: updated {len(self.pc)} critical points")
                return True, (self.pc, PointWrapper(yc_u, self.pc.get_map()), self.edges_z_list, self.indexes)
            else: ## NOTE: there may be something more clever that we can do, like filtering edges etc
                self.logger.info(f"(-1) FAIL: lost some critical points")
        

        self.logger.info(f'=== Recomputing the graph ===')
        self.edges_z_list, self.indexes = None, None ## Clear the edges and indexes to make sure previous ones don't leak through

        ## (0) Get critical point candidates
        with torch.no_grad():
            pc_start = self._get_grid_starting_pts(self.x_grid, self.x_grid_dist)
            self.logger.info(f'(0) SUCCESS - PC Candidates: {len(pc_start)}')

            if self.pc is not None: ## Add previous points to initial seeds
                pc_start = PointWrapper.merge(pc_start, self.pc) ## TODO: when optimizing, we should treat previous CPs with smaller learning rate. make two parameter groups
                self.pc = None ## Clear the CPs to make sure previous ones don't leak through
        self.mpm.plot(self.plotter.plot_function_and_cps, 'plot_start_points_to_find_cps',
                                        [pc_start.cpu().numpy(), "(0) Starting points to find CPs"])
        self.logger.info(f'(0) SUCCESS')
        
        ## (1) Get critical point candidates
        with self.record_time('cp_helper._gd_to_min_grad_norm'):
            pc_candidates, yc_x_norms, x_path_over_iters = self._gd_to_min_grad_norm(z, pc_start, epoch, self.config['lr_find_cps'], early_stop=True, plot_descent=self.plotter.do_plot('plot_cp_descent_recomputation'))
        if self.plotter.do_plot('plot_cp_descent_recomputation'):
            self.mpm.plot(self.plotter.plot_descent_trajectories, 'plot_cp_descent_recomputation',
                                      [pc_start.detach().cpu().numpy(), x_path_over_iters.cpu().numpy()],
                                      {'fig_label':'Trajectories of recomputing CPs'})
                
        self.logger.info(f'(0) POST grad norm - PC Candidates: {len(pc_start)}')
        if len(pc_candidates) == 0:
            self.logger.info(f'(1) MY FAIL: did not find any cp candidates')
            FAILS['(1) MY FAIL: did not find any cp candidates'] += 1
            return False, None
        self.mpm.plot(self.plotter.plot_function_and_cps, 'plot_cp_candidates',
                                      [pc_candidates.detach().cpu().numpy(), "(1) Critical point candidates"],
                                        {'xc_attributes':yc_x_norms.cpu().numpy()})
        
        self.logger.info(f'(1) SUCCESS')

        ## (2) Cluster critical point candidates
        with self.record_time('cp_helper._cluster_cp_candidates'):
            success, pc = self._cluster_cp_candidates(z,pc_candidates, yc_x_norms, **kwargs)
        if not success:
            self.logger.info(f'(2) FAIL: no good clusters')
            FAILS['(2) FAIL: no good clusters'] += 1
            return False, None
        self.pc = pc ## update CPs even if building edges fails since we can reuse them for tracking
        self.logger.info('(2) SUCCESS')
        self.mpm.plot(self.plotter.plot_function_and_cps, 'plot_cps',
                                      [pc.cpu().numpy(), "(2) Critical points after clustering"],
                                      {'p_backdrop':pc_candidates.cpu()})
        self.logger.info(f'(2) POST clustering - PC Clusters: {len(pc)}')

        
        ## (3) Characterize critical points
        with self.record_time('cp_helper._characterize_critical_points'):
            eigvals, eigvecs, indexes, saddle_mask = self._characterize_critical_points(z, pc)
        self.mpm.plot(self.plotter.plot_characterized_cps, 'plot_characterized_cps',
                                      [pc.cpu(), eigvecs.cpu(), eigvals.cpu()])
        if (indexes == 1).sum() == 0:
            self.logger.info(f'(3) FAIL: no saddle points to connect from')
            FAILS['(3) FAIL: no saddle points to connect from'] += 1
            return False, None
        self.logger.info('(3) SUCCESS')

        self.logger.info(f'(3) saddle points {(indexes == 1).sum() == 0}')


        ## (4) Perturb saddle points
        with self.record_time('cp_helper._perturb_saddle_points'):
            p_path0_list, multipliers_to_descend = self._perturb_saddle_points(eigvals, eigvecs, indexes, pc)
        self.logger.info('(4) SUCCESS')
        
        ## (5) Flow from saddle points
        with self.record_time('cp_helper._let_perturbed_points_flow_to_minima'):
            p_path_list, x_path_over_iters = self._let_perturbed_points_flow_to_minima(z, p_path0_list, multipliers_to_descend, epoch)
        if self.plotter.do_plot('plot_saddle_descent'):
            self.mpm.plot(self.plotter.plot_saddle_trajectories, 'plot_saddle_descent', 
                                      [pc.cpu().numpy(), [p.cpu().numpy() for p in p_path_list], x_path_over_iters.cpu().numpy(), 
                                       indexes.cpu().numpy(), multipliers_to_descend.cpu().numpy()])
        self.logger.info('(5) SUCCESS')

        ## (6) Get edges from trajectories
        with self.record_time('cp_helper._get_edges_from_saddle_point_converges'):
            edge_z_list = self._get_edges_from_saddle_point_converges(pc, p_path_list, saddle_mask)
        self.logger.info('(6) SUCCESS')

        with torch.no_grad():
            yc_u = self.netp.f(pc.data, pc.z_in(z)).squeeze(1)
        ## Save to object
        self.pc, self.edges_z_list, self.indexes = pc, edge_z_list, indexes
        
        self.logger.info(f'(6) SUCCESS {len(self.pc)}')

        pyc_u = PointWrapper(yc_u, pc.get_map())
        return True, (pc, pyc_u, edge_z_list, indexes)

    def _gd_to_min_grad_norm(self, z, pc: PointWrapper, lr, epoch=None, early_stop=False, plot_descent=False, **kwargs):
        """
        Find the critical points of the NN by looking for ||grad f|| = 0.
        This is achieved using gradient descent where the points are optimized and the model is frozen.
        xc: [(bz bx) nx]
        """
        # default values
        max_iter = self.config['max_iter_find_cps']
        x_path_over_iters = None
            
        if plot_descent:
            ## for plotting only
            traj_every_nth_iter = self.config['plot_trajectories_every_n_iter']
            x_path_over_iters = torch.zeros([self.config['max_iter_find_cps'] // traj_every_nth_iter + 1, len(pc), self.config['nx']])
            x_path_over_iters.label = f'x_path in CP descent (epoch {epoch})'

        # work directly with the tensor
        pc.data.requires_grad = True  # to optimize xc, not the model self.params

        opt = torch.optim.Adam([pc.data], lr=lr)

        pc_in = None
        for i in (pbar := trange(max_iter)):
            opt.zero_grad()
            
            if plot_descent and (i % traj_every_nth_iter == 0):
                x_path_over_iters[i // traj_every_nth_iter] = pc.data.detach()
                        
            ## Mask the points that have left the domain
            # out_mask = (pc.data < self.bounds[:, 0]).any(1) | (pc.data > self.bounds[:, 1]).any(1)
            out_mask = get_is_out_mask(pc.data, self.bounds)
            stop_mask = out_mask

            pc_in = pc.select_w_mask(incl_mask=~stop_mask)
            self.logger.info(str(pc.data))
            self.logger.info(str(torch.max(pc.data)))
                
            # self.logger.info(f'pc_data {str(pc.data.shape)}')
            # self.logger.info(f'pc_in {pc.data.shape}')
            if len(pc_in) == 0:
                self.logger.info(f'early stop at it {i}: all points left the domain')
                FAILS[f'early stop at it: all points left the domain'] += 1
                break
            ## Compute the gradient norms
            y_x = self.netp.vf_x(pc_in.data, pc_in.z_in(z))  ## [k, ny, (nx+nz)]
            squared_norms = y_x.square().sum(2).sum(1)
            
            ## Check if we can stop early
            norms_small_enough_mask = squared_norms < self.config['dbscan.y_x_mag'] ** 2
            if early_stop and norms_small_enough_mask.all():
                self.logger.info(f'early stop at it {i}: norms_small_enough_mask all true')
                FAILS[f'early stop at it: norms_small_enough_mask all true'] += 1
                break

            ## Update
            loss = squared_norms.mean()  ## compute mean of squared norms
            loss.backward()
            opt.step()
                
            pbar.set_description(f"Descending to CPs: {loss.item():.2e}")
        pbar.close() ## close tqdm which interferes with next print line due to async

        if plot_descent:
            x_path_over_iters = x_path_over_iters[:i // traj_every_nth_iter + 1] ## remove the unfilled part due to early stopping
            x_path_over_iters[-1] = pc.data.detach()
    
        with torch.no_grad():
            # TODO: something seems to be off here: we recompute xc but feed xc_in to compute y_x -
            #  probably we can save recomputing squared_norms since we anyways take the value of the last it
            self.logger.debug(f'found {len(pc_in)} candidates in {i} iterations ')
            # xc = xc[~out_mask].detach()
            # update pc
            pc = pc.select_w_mask(incl_mask=~stop_mask)
            pc = pc.detach()

            if len(pc) > 0:
                y_x = self.netp.vf_x(pc_in.data, pc_in.z_in(z))  ## [bx, ny, nx]
                squared_norms = y_x.square().sum(2).sum(1)

            return pc, squared_norms.detach().sqrt(), x_path_over_iters

    def _cluster_cp_candidates(self, z, pc: PointWrapper, y_x_mag=None, **kwargs):
        """Cluster critical point candidates. Currently uses DBSCAN on CPU."""

        ## (1) Remove all points outside the domain
        out_mask = get_is_out_mask(pc.data, self.bounds)
        # pc_ = xc[~out_mask]
        pc_ = pc.select_w_mask(incl_mask=~out_mask)
        self.logger.debug(f"nof points inside bounds: {len(pc_)}")
        if len(pc_) == 0:
            self.logger.info(f"(-1) MY FAIL: _cluster_cp_candidates 1")
            FAILS["(-1) MY FAIL: _cluster_cp_candidates 1"] += 1
            return False, None
        
        # ## (2) Remove all points that have not converged to critical points within some threshold
        if y_x_mag is not None:
            y_x_mag = y_x_mag[~out_mask]
        else:
            with torch.no_grad():
                y_x = self.netp.vf_x(pc_.data, pc_.z_in(z))
            y_x_mag = y_x.square().sum(2).sum(1).sqrt()

        # move to CPU
        y_x_mag = y_x_mag.detach().cpu()
        pc_ = pc_.cpu()
        # plot histogram of gradient magnitudes
        self.mpm.plot(self.plotter.plot_hist, 'plot_grad_mag_hist', 
                                    [y_x_mag, "Histogram of CPs Gradient Magnitudes"], dont_parallelize=True)

        self.logger.info(f"y_x_mag" + str(y_x_mag.shape) + str(y_x_mag))
        mask = y_x_mag < self.config['dbscan.y_x_mag']
        # pc_ = pc_[mask]
        y_x_mag_ = y_x_mag[mask]
        pc_ = pc_.select_w_mask(incl_mask=mask)
        self.logger.info(f"nof points with small enough gradient: {len(pc_)}")
        if len(pc_) == 0:
            self.logger.info(f"no points with small enough gradient")
            FAILS["no points with small enough gradient"] += 1
            return False, None

        pts_per_shape_list = []
        for i_shape in range(self.config['batch_size']):
            ## (3) Cluster the remaining points and take representative
            xc = pc_.pts_of_shape(i_shape)
            y_x_mag_shape = y_x_mag_[pc_.get_idcs(i_shape)]
            if len(xc) == 0:
                pts_per_shape_list.append(torch.zeros(0))
                continue
                
            dbscan = DBSCAN(eps=self.config['dbscan.eps'], min_samples=1)
            labels = dbscan.fit_predict(xc)
            ## Old: take any instance from the cluster
            # _, idx = np.unique(labels, return_index=True)
            # xc_u = pc_[idx]
            ## New: take the instance with the smallest gradient
            N_clusters = labels.max().item() + 1
            self.logger.debug(f"nof clusters: {N_clusters}")
            smallest_grad_per_cluster = torch.full([N_clusters, ], torch.inf)
            xc_u = torch.zeros([N_clusters, self.config['nx']])
            for i, label in enumerate(labels):
                if y_x_mag_shape[i] < smallest_grad_per_cluster[label]:
                    smallest_grad_per_cluster[label] = y_x_mag_shape[i]
                    xc_u[label] = xc[i]
            pts_per_shape_list.append(xc_u)
        pc_u = PointWrapper.create_from_pts_per_shape_list(pts_per_shape_list)
        pc_u.data.to(pc.data.device)  # TODO: is this really necessary? Should be on default device already, right?
        return True, pc_u

    def _let_perturbed_points_flow_to_minima(self, z, p_path0_list:list[PointWrapper], multipliers_to_descend:torch.Tensor, epoch):
        """A simple optimization loop to let critical points flow along gradient to minima/maxima"""

        # TODO: was cloning here necessary?
        # x_path = p_path0.clone().detach()
        # p_path = PointWrapper(p_path0.data.clone().detach(), map=p_path0.get_map())
        # nof_half_paths = len(p_path) // 2  # index for selecting min/max paths; number of paths to min/max if half of all paths

        x_path = torch.cat([p.data for p in p_path0_list], dim=0)
        z_path = torch.cat([p.z_in(z) for p in p_path0_list], dim=0)
        # nof_half_paths = len(x_path) // 2  # index for selecting min/max paths; number of paths to min/max if half of all paths
        
        x_path.requires_grad = True
        opt = torch.optim.Adam([x_path], lr=self.config['lr_saddle_descent'])

        plot_saddle_descent = self.plotter.do_plot('plot_saddle_descent')
        x_path_over_iters = None
        if plot_saddle_descent:
            traj_every_nth_iter = self.config['plot_trajectories_every_n_iter']
            x_path_over_iters = torch.zeros(
                [self.config['n_iter_saddle_descent'] // traj_every_nth_iter + 1, len(x_path), self.config['nx']])  ## for plotting only
            # y_path_over_iters = torch.zeros([N_iter + 1, len(x_path), ])  ## for plotting only
            x_path_over_iters.label = f'x_path in saddle descent (epoch {epoch})'

        for i in (pbar := trange(self.config['n_iter_saddle_descent'], leave=True, ncols=80, position=0)):
            opt.zero_grad()
            y = self.netp.f(x_path, z_path).squeeze(1)  ## [bx, ny]

            ## for plotting only
            if plot_saddle_descent and (i % traj_every_nth_iter == 0):
                x_path_over_iters[i // traj_every_nth_iter] = x_path.detach()

            loss = (multipliers_to_descend * y).mean()
            loss.backward()
            opt.step()
            pbar.set_description(f"Connecting CPs: {loss.item():.2e}")

            if i % self.config['check_interval_grad_mag_saddle_descent'] == 0:
                # stop if all grad_mags < thresh
                grad_mags = x_path.grad.square().sum(1).sqrt()
                if (grad_mags > self.config['stop_grad_mag_saddle_descent']).sum() == 0:
                    self.logger.debug(f'Early stop "Connecting CPs" at it {i}')
                    break

        ## for plotting only
        if plot_saddle_descent:
            x_path_over_iters = x_path_over_iters[:i // traj_every_nth_iter + 1]
            x_path_over_iters[-1] = x_path.detach()

        p_map = p_path0_list[0].get_map()
        n_pts = len(p_path0_list[0])
        x_path = x_path.detach()
        p_path_list = [PointWrapper(x_path[i*n_pts:(i+1)*n_pts], p_map) for i in range(2 * self.config['nx'])]

        return p_path_list, x_path_over_iters

    def _characterize_critical_points(self, z, pc):
        # characterize the found critical points
        ## Compute the Hessians at all the critical points
        with torch.no_grad():
            H = self.netp.vf_xx(pc.data, pc.z_in(z)).squeeze(1)
        ## Compute the eigendecomposition of each Hessian
        out = torch.linalg.eigh(H)
        eigvals = out.eigenvalues.detach()
        eigvecs = out.eigenvectors.detach()
        # eigvals = torch.linalg.eigvalsh(H) ## NOTE: this is the numerically stable way of computing eigvals if they are needed for AD
        ## The index of the critical point is the number of negative eigenvalues
        indexes = (eigvals < 0).sum(1)
        saddle_mask = (indexes != 0) & (indexes != self.config['nx'])  ## 0 < index < nx; saddle point
        
        self.logger.debug(f"Index of each critical point: {indexes.tolist()}")
        return eigvals, eigvecs, indexes, saddle_mask

    def _perturb_saddle_points(self, eigvals, eigvecs, indexes, pc):
        """
        TODO: we should specify the stepsize from the function, eg its frequency or Lipshitzness
        """

        ## Perturb each saddle point towards + and - of the eigenvector corresponding to the negative eigenvalue
        saddle_mask = (indexes != 0) & (indexes != self.config['nx'])  ## 0 < index < nx; saddle point
        p_saddle = pc.select_w_mask(saddle_mask)

        eigvals = eigvals[saddle_mask] ## [k, nx]
        eigvecs = eigvecs[saddle_mask] ## [k, nx, ith-vector]  
        eigvecs = eigvecs.transpose(1, 2) ## [k, ith-vector, nx]  # TODO: check if this is correct
        multiplier_to_descend = torch.where(eigvals < 0, 1, -1) ## [k, nx]
        
        multiplier_to_descend_list = []
        p_path0_list = []
        for i in range(eigvals.shape[1]):
            dx = eigvecs[:, i, :] ## [k, nx]
            x_i_pos = p_saddle.data + self.config['perturbation_saddle_descent'] * dx
            x_i_neg = p_saddle.data - self.config['perturbation_saddle_descent'] * dx
            p_path0_list.append(PointWrapper(x_i_pos, map=p_saddle.get_map()))
            p_path0_list.append(PointWrapper(x_i_neg, map=p_saddle.get_map()))
            multiplier_to_descend_list.append(multiplier_to_descend[:, i])
            multiplier_to_descend_list.append(multiplier_to_descend[:, i])
            
        multipliers_to_descend = torch.cat(multiplier_to_descend_list)
        return p_path0_list, multipliers_to_descend

    def _get_edges_from_saddle_point_converges(self, pc, p_path_list:list[PointWrapper], saddle_mask_all_z):

        edges_z_list = []

        for i_shape in range(self.config['batch_size']):
            if len(pc.pts_of_shape(i_shape)) == 0:
                edges_z_list.append(torch.zeros(0))
                continue
            x_path = torch.vstack([p.pts_of_shape(i_shape) for p in p_path_list]) ## [nx*bx, nx]
            xc = pc.pts_of_shape(i_shape)
            saddle_mask = saddle_mask_all_z[pc.get_idcs(i_shape)]

            ## We need to find to which extreme point we converged
            d = (x_path[:, None, :] - xc[None, :, :]).square().sum(2).sqrt()  ## NOTE: we only have to measure to extreme points, so mask saddle points.
            ## NOTE: mby keep track of different xc lists depending on their index
            attractors = d.min(1)
            idxs_of_crits = attractors.indices
            dists_to_crits = attractors.values
            idxs_of_saddles_in_u = torch.where(saddle_mask)[0]
            edges = torch.stack([idxs_of_saddles_in_u.repeat(2 * self.config['nx']), idxs_of_crits]).T
            # self.logger.debug(edges)
            ## For each edge, check if the path actually converged. TODO: we don't know what the exact best approach is
            ## (1) Remove all points outside the domain
            ## (2) Remove all points that have not converged to critical points within some threshold
            ## (3) Remove all points which are not close enough to known critical point
            dist_mask = dists_to_crits < self.config['dbscan.eps']
            edges = edges[dist_mask]
            ## TODO: Eg if we have converged to an extreme point (||grad||<eps), but there is no known point there, we have missed it
            edges_z_list.append(edges)

        return edges_z_list
            
    def _get_grid_starting_pts(self, x_grid, grid_dist):
        '''
        Create grid once at beginning.
        Translate the grid by a random offset.
        '''
        xc_offset = torch.rand((self.config['batch_size'], self.config['nx'])) * grid_dist  # bz nx
        # x_grid: [n_points nx]
        x = x_grid.unsqueeze(0) + xc_offset.unsqueeze(1)  # bz n_points nx
        return PointWrapper.create_from_equal_bx(x)

In [5]:
import logging
import torch

# from GINN.morse.cp_helper import CriticalPointsHelper
from GINN.morse.surfacegraph_helper import SurfaceGraphHelper
from utils import set_and_true

logging.basicConfig(level=logging.INFO)

class SCCSurfaceNetManager:

    def __init__(self, config, netp, mp_manager, plotter, timer_helper, p_sampler, device):
        self.logger = logging.getLogger('SCCSurfaceNetManager')
        self.config = config
        self.mpm = mp_manager
        self.plotter = plotter
        self.timer_helper = timer_helper
        self.p_sampler = p_sampler
        self.device = device
        
        self.cp_helper = CriticalPointsHelper(config, netp, mp_manager, plotter, timer_helper, device)
        self.surfgraph_helper = SurfaceGraphHelper(self.config, self.mpm, self.plotter, self.timer_helper)
        

    def get_surface_pts(self, z, epoch):
        success, p_surface = self.cp_helper.get_surface_pts(z, epoch)
        if not success:
            return None, None

        weights_surf_pts = torch.ones(len(p_surface)) / p_surface.data.shape[0]
        if set_and_true('reweigh_surface_pts_close_to_interface', self.config):
            dist = torch.min(torch.norm(p_surface.data[:, None, :] - self.x_interface[None, :, :], dim=2), dim=1)[0]
            dist = torch.clamp(dist, max=self.config['reweigh_surface_pts_close_to_interface_cutoff'])
            weights_surf_pts = torch.pow(dist, self.config['reweigh_surface_pts_close_to_interface_power'])
            weights_surf_pts = weights_surf_pts / weights_surf_pts.sum()  ## normalize to sum to 1

        self.mpm.plot(self.plotter.plot_shape_and_points, 'plot_surface_points', arg_list=[p_surface.detach().cpu(), 'Surface points'])
        return p_surface, weights_surf_pts

    def get_scc_pts_to_penalize(self, z, epoch):
        ## Critical points
        success, res_tup = self.cp_helper.get_critical_points(z, epoch, plot_helper=self.plotter,
                                                              timer_helper=self.timer_helper)
        if not success:
            self.logger.info("get_critical_points failed")
            FAILS["get_critical_points failed"] += 1
            return False, None

        (pc, pyc, edges_list, indexes) = res_tup
        # move to cpu
        pc_cpu, pyc_cpu, indexes_cpu = pc.cpu(), pyc.cpu(), indexes.cpu()
        edges_list = [edges.cpu() for edges in edges_list]
        indexes_list = [indexes_cpu[pc_cpu.get_idcs(i_shape)] for i_shape in range(self.config['batch_size'])]

        self.mpm.plot(self.plotter.plot_edges,'plot_edges', arg_list=[pc_cpu, indexes_list, edges_list, "Surface graph edges"])

        ## Surface graphs
        surfnet_graph_list = []
        with self.timer_helper.record('surfgraph_helper.construct_surfnet_graph'):
            for edges_cpu in edges_list:
                surfnet_graph_list.append(self.surfgraph_helper.construct_surface_net(edges_cpu, pc_cpu))

        with self.timer_helper.record('surfgraph_helper.get_sub_CC'):
            surfnet_graph_sub_CCs_list = self.surfgraph_helper.get_sub_CC(edges_list, pyc_cpu)
        self.mpm.plot(self.plotter.plot_CCs_of_surfnet_graph, 'plot_surfnet_graph', arg_list=[surfnet_graph_sub_CCs_list, pc_cpu.numpy(), pyc_cpu.numpy(), edges_list])

        with self.timer_helper.record('surfgraph_helper.get_cps_and_penalties_between_CCs'):
            p_penalize, p_penalties = self.surfgraph_helper.get_cps_and_penalties_between_CCs_list(surfnet_graph_list,
                                                                                              surfnet_graph_sub_CCs_list,
                                                                                              pc_cpu,
                                                                                              indexes_cpu, pyc_cpu,
                                                                                              plot_helper=self.plotter)

        if p_penalties.data.sum() == 0:
            self.logger.info("Not enough p_penalties sum")
            FAILS["Not enough p_penalties sum"] += 1
            return False, None

        self.mpm.plot(self.plotter.plot_shape_and_points, 'plot_penalty_points', arg_list=[p_penalize.cpu().numpy(), 'Penalty points'])

        p_penalize = p_penalize.to(self.device)
        p_penalties = p_penalties.to(self.device)

        if set_and_true('remove_penalty_points_outside_envelope', self.config):
            is_in_env_mask = self.p_sampler.is_inside_envelope(p_penalize.cpu().numpy())
            is_in_env_mask = torch.from_numpy(is_in_env_mask).to(torch.bool).to(self.device)
            self.logger.debug(
                f'penalize DCs with {is_in_env_mask.sum()} points inside envelope and {(~is_in_env_mask).sum()} points outside')
            p_penalize = p_penalize.select_w_mask(is_in_env_mask)
            p_penalties = p_penalties.select_w_mask(is_in_env_mask)
            if p_penalize.data.sum() == 0:
                self.logger.debug(f'No points left after removing points outside envelope')
                FAILS["No points left after removing points outside envelope"] += 1
                return False, None

            self.mpm.plot(self.plotter.plot_shape_and_points, 'plot_penalty_points_only_in_envelope',
                                          arg_list=[p_penalize.cpu().numpy(), 'Penalty points w/o points outside envelope'])

        return True, (p_penalize, p_penalties)

## Setup

In [6]:
set_all_seeds(5)
## Set the device
os.environ['CUDA_VISIBLE_DEVICES'] = str(cuda_device)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.set_default_device(device)

## Read the config
config = get_config_from_yml(yml_path)
config['device'] = device

print("DEVICE", device)
print("MODEL:", config['model'])
print("ACTIVATION", config.get('activation', None))
print("PROBLEM:", config['problem'])

print(config['dbscan.eps'])
print(config['dbscan.y_x_mag'])

## Create the model and stateless functions and load a checkpoint
model = get_model(config)
netp = get_stateless_net_with_partials(model, use_x_and_z_arg=config['use_x_and_z_arg'])
# model.load_state_dict(torch.load('_quickstart/trained_model_3d.pt', map_location=device))

## Create different helpers for ...
## ... the problem definition
p_sampler = ProblemSampler(config)
print(p_sampler.constr_pts_dict['domain'].shape)
print(p_sampler.constr_pts_dict['envelope'].shape)

mp_manager = MPManager(config)
timer_helper = TimerHelper(config, lock=mp_manager.get_lock())
mp_manager.set_timer_helper(timer_helper)  ## weak circular reference
plotter = Plotter3d(config)
scc_manager = SCCSurfaceNetManager(config, netp, mp_manager, plotter, timer_helper, p_sampler, device)
shapeb_helper = ShapeBoundaryHelper(config, netp, mp_manager, plotter, timer_helper, p_sampler.sample_from_interface()[0], device)
auto_clip = AutoClip(config)

DEVICE cuda
MODEL: cond_siren
ACTIVATION None
PROBLEM: simjeb
0.05
1e-05
(2048, 3)
(5461, 3)


## Plot the checkpoint

In [7]:
# from IPython.display import Image

# z = torch.tensor([-0.1])
# mesh_checkpoint = get_mesh_for_latent(netp.f_, netp.params_, z, config['bounds'], mc_resolution=64, device=device, chunks=1)

# fig = k3d.plot()
# fig += k3d.mesh(*mesh_checkpoint, color=0xff0000, side='double')

# fig.display()

# fig.camera_auto_fit = False
# fig.camera = [0.8042741481976844,
#             -1.040350835893895,
#             0.7038650223301532,
#             0.08252720725551285,
#             -0.08146462547370059,
#             -0.1973267630672968,
#             -0.3986658507677483,
#             0.39231188503442904,
#             0.8289492893370278]

# # fig.display()

## Train

In [8]:
from torch.utils.tensorboard import SummaryWriter
import datetime
import os
import yaml

log_dir = os.path.join(runs_dir, "runs_quickstart", datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "_" + filename)
writer = SummaryWriter(log_dir)

opt = torch.optim.Adam(model.parameters(), lr=config['lr'])
# opt.load_state_dict(torch.load('_quickstart/opt_3d.pt', map_location=device))
z = sample_new_z(config, is_init=True)

print(f'Initial z: {z}')
p_surface = None
cur_plot_epoch = 0
plots_dict_at_last_epoch = None
log_history_dict = {}

prev_epoch = 0

Initial z: tensor([[-0.1000]], device='cuda:0')


In [9]:
print(config['bounds'])

config['max_epochs'] = num_epochs
# config['bounds'] = torch.Tensor([[-2, 2], [-2, 2], [-2, 2]])
total_loss_scc = 0

with open(os.path.join(log_dir, "config.yml"), 'w') as file: 
    yaml.dump(config, file)

for epoch in (pbar := trange(config['max_epochs'], leave=True, position=0, colour="yellow")):
    # epoch += prev_epoch

    mp_manager.update_epoch(epoch)
    opt.zero_grad()
    
    plotter.reset_output(p_sampler.recalc_output(netp.f_, netp.params_, z), epoch=epoch)
    plotter.plot_shape(p_sampler.constr_pts_dict)
    
    loss_scc = torch.tensor(0.0)
    if config['lambda_scc'] > 0:
        success, res_tup = scc_manager.get_scc_pts_to_penalize(z, epoch)
        if success:
            p_penalize, p_penalties = res_tup
            print(f'penalize DCs with {len(p_penalize)} points')
            y_saddles_opt = model(p_penalize.data, p_penalize.z_in(z)).squeeze(1)
            loss_scc = config['lambda_scc'] *  (y_saddles_opt * p_penalties.data).mean()
            print("my scc loss:", loss_scc)
        else:
            FAILS["get_scc_pts_to_penalize failed"] += 1

    ## Design region loss                
    loss_env = torch.tensor(0.0)
    if config['lambda_env'] > 0:
        ys_env = model(*tensor_product_xz(p_sampler.sample_from_envelope(), z)).squeeze(1)
        loss_env = config['lambda_env'] * envelope_loss(ys_env)

    ## Interface loss
    loss_if = torch.tensor(0.0)
    if config['lambda_bc'] > 0:
        ys_BC = model(*tensor_product_xz(p_sampler.sample_from_interface()[0], z)).squeeze(1)
        loss_if = config['lambda_bc'] * interface_loss(ys_BC)
        
    ## Interface normal loss
    loss_if_normal = torch.tensor(0.0)
    if config['lambda_normal'] > 0:
        pts_normal, target_normal = p_sampler.sample_from_interface()
        ys_normal = netp.vf_x(*tensor_product_xz(pts_normal, z)).squeeze(1)
        loss_if_normal = config['lambda_normal'] * normal_loss_euclidean(ys_normal, torch.cat([target_normal for _ in range(config['batch_size'])]))

    ## Obstacle loss (for debugging purposes, it's not considered part of the envelope) TODO: do we leave it like this?
    loss_obst = torch.tensor(0.0)
    if config['lambda_obst'] > 0:
        ys_obst = model(*tensor_product_xz(p_sampler.sample_from_obstacles(), z))
        loss_obst = config['lambda_obst'] * obstacle_interior_loss(ys_obst)

    ## Sample points from the domain if necessary TODO: I think diversity doesnt need domain points anymore? TODO: can we move this up so that all the losses come after each other?
    if config['lambda_eikonal'] > 0 or config['lambda_div'] > 0:
        xs_domain = p_sampler.sample_from_domain()

    ## Eikonal loss    
    loss_eikonal = torch.tensor(0.0)
    if config['lambda_eikonal'] > 0:
        y_x_eikonal = netp.vf_x(*tensor_product_xz(xs_domain, z))
        loss_eikonal = config['lambda_eikonal'] * eikonal_loss(y_x_eikonal)

    ## Sample points from the 0-levelset if necessary TODO: can we move this up?
    if config['lambda_div'] > 0 or config['lambda_curv'] > 0:
        if p_surface is None or epoch % config['recompute_surface_pts_every_n_epochs'] == 0:
            p_surface, weights_surf_pts = shapeb_helper.get_surface_pts(z)
    
    ## Curvature loss
    loss_curv = torch.tensor(0.0)
    if config['lambda_curv'] > 0:
        if p_surface is None:
            print('No surface points found - skipping curvature loss')
        else:
            y_x_surf = netp.vf_x(p_surface.data, p_surface.z_in(z)).squeeze(1)
            y_xx_surf = netp.vf_xx(p_surface.data, p_surface.z_in(z)).squeeze(1)
            loss_curv = config['lambda_curv'] * strain_curvature_loss(y_x_surf, y_xx_surf, clip_max_value=config['strain_curvature_clip_max'],
                                                                            weights=weights_surf_pts)
    ## Diversity loss
    loss_div = torch.tensor(0.0)
    if config['lambda_div'] > 0 and config['batch_size'] > 1:
        if p_surface is None:
            print('No surface points found - skipping diversity loss')
        else:
            y_div = model(*tensor_product_xz(p_surface.data, z)).squeeze(1)  # [(bz k)] whereas k is n_surface_points; evaluate model at all surface points for each shape
            loss_div = config['lambda_div'] * closest_shape_diversity_loss(einops.rearrange(y_div, '(bz k)-> bz k', bz=config['batch_size']), 
                                                                                weights=weights_surf_pts)
            if torch.isnan(loss_div) or torch.isinf(loss_div):
                print(f'NaN or Inf loss_div: {loss_div}')
                loss_div = torch.tensor(0.0) if torch.isnan(loss_div) or torch.isinf(loss_div) else loss_div 

    loss = loss_env + loss_if + loss_if_normal + loss_obst + loss_eikonal + loss_scc + loss_curv + loss_div
    
    losses = {
        "loss_env": loss_env,
        "loss_if": loss_if,
        "loss_if_normal": loss_if_normal,
        "loss_obst": loss_obst,
        "loss_eikonal": loss_eikonal,
        "loss_scc": loss_scc,
        "loss_curv": loss_curv,
        "loss_div": loss_div,
        "loss": loss,
    }

    ## Gradients with clipping
    loss.backward()
    grad_norm = auto_clip.grad_norm(model.parameters())
    if auto_clip.grad_clip_enabled:
        auto_clip.update_gradient_norm_history(grad_norm)
        torch.nn.utils.clip_grad_norm_(model.parameters(), auto_clip.get_clip_value())
        
    ## Update the parameters
    opt.step()

    for lname, l in losses.items():
        writer.add_scalar(f'Loss/{lname}', l, epoch)

    for name, param in model.named_parameters():
        writer.add_histogram(f'Weights/{name}', param, epoch)
        if param.grad is not None:
            writer.add_histogram(f'Gradients/{name}', param.grad, epoch)
    
    if (epoch % save_interval == 0 and epoch > 0) or (epoch - prev_epoch - 1) == config['max_epochs']:
        save_data = {
            # 'f_': netp.f_,              # Assuming this is a model or callable
            'params_': netp.params_,    # Model parameters or some state dict
            'z': z,                      # Latent tensor
            'bounds': config['bounds']
        }
        save_path_pts = os.path.join(log_dir, "pts")
        if not os.path.exists(save_path_pts): os.makedirs(save_path_pts)
        save_path = os.path.join(save_path_pts, f"nept_{epoch}.pt")
        torch.save(save_data, save_path)
    
    # Look at debugging plots
    # For this you have to enable plots in the config; note: this will slow down the training
    if mp_manager.are_plots_available_for_epoch(epoch):
        plots_dict_at_last_epoch = mp_manager.pop_plots_dict(epoch)

    total_loss_scc += loss_scc.item()

    print(FAILS)
    print("================================================")

prev_epoch += config['max_epochs']
print(total_loss_scc)

tensor([[-0.5914,  0.5914],
        [-1.0000,  1.0000],
        [-0.3469,  0.3469]], device='cuda:0')


  0%|          | 0/100 [00:00<?, ?it/s]INFO:cp_helper:=== Recomputing the graph ===
INFO:cp_helper:(0) SUCCESS - PC Candidates: 3375
INFO:cp_helper:(0) SUCCESS
INFO:cp_helper:tensor([[-0.5334, -0.9094, -0.2954],
        [-0.5334, -0.9094, -0.2520],
        [-0.5334, -0.9094, -0.2086],
        ...,
        [ 0.5016,  0.8406,  0.2250],
        [ 0.5016,  0.8406,  0.2684],
        [ 0.5016,  0.8406,  0.3117]], device='cuda:0', requires_grad=True)
INFO:cp_helper:tensor(0.8406, device='cuda:0', grad_fn=<MaxBackward1>)
INFO:cp_helper:tensor([[-0.5334, -0.9094, -0.2954],
        [-0.5334, -0.9094, -0.2520],
        [-0.5334, -0.9094, -0.2086],
        ...,
        [ 0.5016,  0.8406,  0.2250],
        [ 0.5016,  0.8406,  0.2684],
        [ 0.5016,  0.8406,  0.3117]], device='cuda:0', requires_grad=True)
INFO:cp_helper:tensor(0.8406, device='cuda:0', grad_fn=<MaxBackward1>)
INFO:cp_helper:tensor([[-0.5334, -0.9094, -0.2954],
        [-0.5334, -0.9094, -0.2520],
        [-0.5334, -0.9094, -0.208

Counter({'no points with small enough gradient': 1, '(2) FAIL: no good clusters': 1, 'get_critical_points failed': 1, 'get_scc_pts_to_penalize failed': 1})


INFO:cp_helper:=== Recomputing the graph ===
INFO:cp_helper:(0) SUCCESS - PC Candidates: 3375
INFO:cp_helper:(0) SUCCESS
INFO:cp_helper:tensor([[-0.4956, -0.8736, -0.2923],
        [-0.4956, -0.8736, -0.2489],
        [-0.4956, -0.8736, -0.2055],
        ...,
        [ 0.5394,  0.8764,  0.2281],
        [ 0.5394,  0.8764,  0.2715],
        [ 0.5394,  0.8764,  0.3148]], device='cuda:0', requires_grad=True)
INFO:cp_helper:tensor(0.8764, device='cuda:0', grad_fn=<MaxBackward1>)
INFO:cp_helper:tensor([[ 0.5044,  0.1264,  0.7077],
        [ 0.5044,  0.1264, -1.2489],
        [ 0.5044, -1.8736, -1.2055],
        ...,
        [ 1.5394,  1.8764, -0.7719],
        [-0.4606,  1.8764,  1.2715],
        [ 1.5394,  1.8764,  1.3148]], device='cuda:0', requires_grad=True)
INFO:cp_helper:tensor(1.8764, device='cuda:0', grad_fn=<MaxBackward1>)
INFO:cp_helper:early stop at it 1: all points left the domain
Descending to CPs: 1.85e+02:   0%|          | 1/1000 [00:00<00:27, 36.55it/s]
INFO:cp_helper:(0) PO

Counter({'get_critical_points failed': 2, 'get_scc_pts_to_penalize failed': 2, 'no points with small enough gradient': 1, '(2) FAIL: no good clusters': 1, 'early stop at it: all points left the domain': 1, '(1) MY FAIL: did not find any cp candidates': 1})


...

Counter({'get_critical_points failed': 98, 'get_scc_pts_to_penalize failed': 98, 'early stop at it: all points left the domain': 97, '(1) MY FAIL: did not find any cp candidates': 97, 'no points with small enough gradient': 1, '(2) FAIL: no good clusters': 1})


INFO:cp_helper:=== Recomputing the graph ===
INFO:cp_helper:(0) SUCCESS - PC Candidates: 3375
INFO:cp_helper:(0) SUCCESS
INFO:cp_helper:tensor([[-0.5076, -0.8867, -0.3062],
        [-0.5076, -0.8867, -0.2628],
        [-0.5076, -0.8867, -0.2194],
        ...,
        [ 0.5274,  0.8633,  0.2142],
        [ 0.5274,  0.8633,  0.2576],
        [ 0.5274,  0.8633,  0.3009]], device='cuda:0', requires_grad=True)
INFO:cp_helper:tensor(0.8633, device='cuda:0', grad_fn=<MaxBackward1>)
INFO:cp_helper:tensor([[-98.5076, -98.8866,  97.6936],
        [-98.5075, -98.8867,  97.7372],
        [-98.5076,  97.1133,  97.7806],
        ...,
        [ 98.5274, -97.1366, -97.7858],
        [-97.4726, -97.1367,  98.2576],
        [-97.4725, -97.1367,  98.3009]], device='cuda:0', requires_grad=True)
INFO:cp_helper:tensor(98.8633, device='cuda:0', grad_fn=<MaxBackward1>)
INFO:cp_helper:early stop at it 1: all points left the domain
Descending to CPs: 5.18e+01:   0%|          | 1/1000 [00:00<00:11, 85.57it/s]
IN

Counter({'get_critical_points failed': 99, 'get_scc_pts_to_penalize failed': 99, 'early stop at it: all points left the domain': 98, '(1) MY FAIL: did not find any cp candidates': 98, 'no points with small enough gradient': 1, '(2) FAIL: no good clusters': 1})


INFO:cp_helper:=== Recomputing the graph ===
INFO:cp_helper:(0) SUCCESS - PC Candidates: 3375
INFO:cp_helper:(0) SUCCESS
INFO:cp_helper:tensor([[-0.5427, -0.8613, -0.3161],
        [-0.5427, -0.8613, -0.2728],
        [-0.5427, -0.8613, -0.2294],
        ...,
        [ 0.4923,  0.8887,  0.2043],
        [ 0.4923,  0.8887,  0.2476],
        [ 0.4923,  0.8887,  0.2910]], device='cuda:0', requires_grad=True)
INFO:cp_helper:tensor(0.8887, device='cuda:0', grad_fn=<MaxBackward1>)
INFO:cp_helper:tensor([[-99.5427, -99.8613,  98.6839],
        [-99.5427,  98.1386,  98.7272],
        [ 98.4573, -99.8613, -99.2294],
        ...,
        [ 99.4923,  99.8886,  99.2042],
        [-98.5077,  99.8886,  99.2476],
        [-98.5077,  99.8886, -98.7090]], device='cuda:0', requires_grad=True)
INFO:cp_helper:tensor(99.8887, device='cuda:0', grad_fn=<MaxBackward1>)
INFO:cp_helper:early stop at it 1: all points left the domain
Descending to CPs: 5.12e+01:   0%|          | 1/1000 [00:00<00:11, 83.86it/s]
IN

Counter({'get_critical_points failed': 100, 'get_scc_pts_to_penalize failed': 100, 'early stop at it: all points left the domain': 99, '(1) MY FAIL: did not find any cp candidates': 99, 'no points with small enough gradient': 1, '(2) FAIL: no good clusters': 1})
0.0


## Comparing the updated and the old mesh
The old mesh is red, the new one in green. In the "K3D Panel" of the plot you can click on "Objects". If you want to do make the updated shape visible, you can click the "visible" tickbox of the according shape. This way you can see the difference in the updates better.

In [10]:
## NOTE: 8GB is not enough CUDA memory to perform marching cubes after training. Maybe we release some tensors? Alt: I would love to understand what these tensors
## TODO: smaller update?

z = torch.tensor([-0.1])
mesh_update = get_mesh_for_latent(netp.f_, netp.params_, z, config['bounds'], mc_resolution=64, device=device, chunks=1)

fig = k3d.plot()
# fig += k3d.mesh(*mesh_checkpoint, color=0xff0000, side='double', opacity=0.5, name='Original shape')
fig += k3d.mesh(*mesh_update, color=0x00ff00, side='double', opacity=0.5, name='Updated shape')
fig.display()
fig.camera_auto_fit = False
fig.camera = [0.8042741481976844,
            -1.040350835893895,
            0.7038650223301532,
            0.08252720725551285,
            -0.08146462547370059,
            -0.1973267630672968,
            -0.3986658507677483,
            0.39231188503442904,
            0.8289492893370278]

Output()

In [ ]:
print(netp.f_)
print(netp.params_)
print(z)

<function get_stateless_net_with_partials.<locals>.f at 0x7f1fbf6c4540>
{'network.0.weight': Parameter containing:
tensor([[-0.1764,  0.5928, -0.9982, -0.4972],
        [-1.0536,  0.6654, -0.3692,  0.4228],
        [-0.3361, -0.5319, -0.4987,  0.4744],
        ...,
        [ 0.8507,  0.9274, -0.8724, -0.0356],
        [ 0.1901,  0.9983, -0.9900, -0.7003],
        [-0.5282, -1.1935, -0.3789,  0.8665]], device='cuda:0',
       requires_grad=True), 'network.0.bias': Parameter containing:
tensor([-0.1565, -0.2399, -0.2059, -0.1746,  0.2825,  0.2149, -0.4267, -0.2400,
         0.4955, -0.2591,  0.2131,  0.4694,  0.3463,  0.4660,  0.0143,  0.3496,
        -0.2749, -0.4111, -0.1815,  0.1875, -0.2426,  0.1296, -0.1169, -0.2346,
         0.0130,  0.4632,  0.0608,  0.2822,  0.3469,  0.3325, -0.1481,  0.3983,
        -0.1887,  0.2953, -0.2315,  0.0557, -0.3353,  0.1779,  0.3826, -0.4349,
        -0.3924, -0.1529,  0.0159,  0.0676,  0.4645, -0.3827,  0.1755, -0.0086,
        -0.4756, -0.2258,  0.0

## Access plots of the last training epoch

In [ ]:
if plots_dict_at_last_epoch is not None:
    # check if variable is set
    print(f'Available plot keys:')
    for k in sorted(plots_dict_at_last_epoch.keys()):
        print(k)

In [ ]:
if plots_dict_at_last_epoch is not None:
    plot_key = 'Characterized critical points'
    plots_dict_at_last_epoch[plot_key].display()